In [8]:
from PIL import Image, ImageDraw
import os
import collections
from math import floor
import sys

def rems(line):
    line=line.replace(' ','')
    line=line.replace('\t','')
    line=line.replace('\r','')
    line=line.replace('\n','')
    return line

def direct(name_file):
    path=os.getcwd()
    path+='/ignis'
    path+='/'+name_file.split('.')[0]
    try:
        os.mkdir(path) # только на один уровень ниже. иначе исп. makedirs()
    except OSError:
        print('Создать дирректорию %s не удалось. Возможно, она уже была создана.' % path)
    else:
        print('Успешно создана дирректория %s' % path)
    return path

def ren32(path_pic,path,num,metka):
    try:
        image=Image.open(path_pic+metka+'.bmp')
        image.save(path+'/'+'{:0>2}_{}.bmp'.format(num,metka), "BMP")
    except FileNotFoundError:
        print()
        print('НЕТ ФАЙЛА %s \n' %(path_pic+metka+'.bmp'))
        
def ren64(path_pic,path,num,metka):
    image1 = Image.open(path_pic+metka+'.bmp') #Открываем изображение. 
    image2=Image.open(path_pic+'white_64.bmp')
    width = image1.size[0] #Определяем ширину. 
    height1 = image1.size[1] #Определяем высоту.
    print(path_pic+metka+'.bmp')
    height2 = image2.size[1] #Определяем высоту.
    draw = ImageDraw.Draw(image2) #Создаем инструмент для рисования. 
    pix = image1.load()
    print(pix)

    for i in range(width):
        k=-1
        for j in range(height1):
            a = pix[i, j][0]
            b = pix[i, j][1]
            c = pix[i, j][2]
            k+=1
            draw.point((i, k), (a, b, c))
            k+=1
            draw.point((i, k), (a, b, c))

    image2.save(path+'/'+'{:0>2}_{}.bmp'.format(num,metka), "BMP")
    del draw
    
SYSTEM_SIZE = 400
print('Не забудь поменять папку с картинками \n')
print('ВВедите имя будущей папки для реквизита. Помните, что оно должно начинаться с  XX_prog_')
print('где XX - порядковый  номер программы у пойки')
name_file_out=input()
#########name_file_out=input()  ################################
path_pic='/home/gatupov/python/ignis_projects/pictures/' ## ОБЯЗАТЕЛЬНО ПОМЕНЯТЬ ПОЛЬЗОВАТЕЛЮ!!!!!!

#######################################################
#name_file_out='04_prog_routable_poi'
path_intervals=os.getcwd()
path_intervals+='/ignis'
path_intervals+='/'+name_file_out.split('.')[0]+'_intervals.txt'
file_intervals=open(path_intervals,'w')

name_path=direct(name_file_out)
file_out=open(name_path+'/program.txt','w')

time=[]
metka=[]
person=[]
full_metki=[]
print('Введите имя дорожки отметок')
name_file=input()
rekv=[]
person_rekv=[]
flag_rekv=0
#name_file='routable_poi_metki.txt'
with open(name_file) as mfile:
    for st in mfile: # проверка
        if not(len(st.split('\t'))==3) and not('#' in st):
            print('DANGER')
        else:
            if ('#' in st):
                line=st.split('-')[1]
                line=line.replace(' ','')
                line=line.replace('\t','')
                line=line.replace('\r','')
                line=line.replace('\n','')
                rekv.append(int(line)) 
                
                line=st.split('-')[0]
                line=line.replace('#','')
                line=line.replace(' ','')
                line=line.replace('\t','')
                line=line.replace('\r','')
                line=line.replace('\n','')
                person_rekv.append(line)
                flag_rekv+=1
            if not('#' in st):   
                time.append(st.split('\t')[1])
                s=st.split('\t')[2]
                full_metki.append(s)
                if '>' in s:
                    if '/' in s:
                        chasti=s.split('/')
                        person_sbor=''
                        metka_sbor=''
                        for ch in chasti:
                            person_sbor+=ch.split('>')[0]+';'
                            metka_sbor+=ch.split('>')[1]+';'
                        person.append(person_sbor[:-1])
                        metka.append(metka_sbor[:-2])
                    else:
                        person.append(s.split('>')[0])
                        metka.append(s.split('>')[1][:-1])
                else:
                    person.append('all')
                    metka.append(s[:-1])
print()
print(metka)
print(time)
print(person_rekv)
print(rekv)



all_persons=[]
for elem in person:
    pers=elem.split(';')
    for p in pers:
        all_persons.append(p)

persons=list(collections.Counter(all_persons).keys())
rekv2=[int(32)]*len(persons)

if flag_rekv>1:
    for j in range(len(persons)):
        for i in range(len(person_rekv)):
            if person_rekv[i]==persons[j]:
                rekv2[j]=rekv[i]
if flag_rekv==1:
    for j in range(len(persons)):
        rekv2[j]=rekv[0]


            
print(persons)
people=0

if len(persons)==1 and persons[0]=='all':
    people=1 # одна прога на всех
    if flag_rekv==0:
        print("Ошибка: вы не указали длину реквизита в пикселях в начале документа")
        sys.exit()

dtime=[]
stime=[]
pictures=[]
intervals=[]
pop=0
for j in range(len(time)):
    dtime.append(( float(time[j]) - float(time[0]) ))
for j in range(1,len(time)):
    intervals.append( ( float(time[j]) - float(time[j-1]) ) )
intervals.append(0)
for j in range(len(time)):
    minn=int(dtime[j]//60)
    sec=int(floor(dtime[j]%60))-pop
    if (sec==-1):
        sec=59
        minn-=1
    msec=int(floor((dtime[j] - floor(dtime[j]%60) - minn*60)*100))
    stime.append('{:0>2}:{:0>2}:{:0>2}'.format(minn,sec,msec))
    pop=1
    
print(stime)

for j in range(len(full_metki)):
    file_intervals.write('{}\t{}\n'.format(full_metki[j][:-1],round(intervals[j],2)))

print('\n #### ЛОГИ #### \n')
#dop='Finish - {}\nRepeat after finish - no\nLock buttons - yes'.format(stime[len(time)-1])

if people==1:
    if flag_rekv==0:
        print("Ошибка: вы не указали длину реквизита в пикселях в начале документа")
        sys.exit()
    
    
    numer=0
    for j in range(len(time)):
        star=0
        dol = 0
        if '*' in metka[j]:
            metka[j] = metka[j].replace('*','')
            star=1
        if '$' in metka[j]:
            metka[j] = metka[j].replace('$','')
            dol=1
        
        if metka[j]=='end':
            dop='Finish - {}\nRepeat after finish - no\nLock buttons - yes'.format(stime[j])
            file_out.write(dop)
        else:
            if not metka[j] in pictures:
                numer+=1
                if rekv[0]==32:
                    ren32(path_pic,name_path,numer,metka[j])
                if rekv[0]==64:
                    ren64(path_pic,name_path,numer,metka[j])
                if not(rekv[0]==32) and not(rekv[0]==64):
                    print('Описка в указании длины реквизита. Неверное число пикселей')
                    print('Допустимое число пикселей - 32 или 64')
                if star==0 and dol==0:
                    file_out.write('{:0>2}_{} - {}\n'.format(numer,metka[j],stime[j]))
                if star==1 and dol==0:
                    speed=round((intervals[j]/SYSTEM_SIZE*1000),1)
                    if speed > 9.9:
                        print('\nРазмер картинки {} слишком мал. Ее длительность {} c. Время {}'.format(metka[j],intervals[j],stime[j]))
                        print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                        file_out.write('{:0>2}_{} - {} ({})\n'.format(numer,metka[j],stime[j],speed))
                    if speed < 0.1:
                        print('\nРазмер картинки {} слишком велик. Ее длительность {} c. Время {}'.format(metka[j],intervals[j],stime[j]))
                        print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                        file_out.write('{:0>2}_{} - {} ({})\n'.format(numer,metka[j],stime[j],speed))
                    if speed >= 0.1 and speed <= 9.9:
                        file_out.write('{:0>2}_{} - {} ({})\n'.format(numer,metka[j],stime[j],speed))
                if star==0 and dol==1:
                    speed=9.9
                    file_out.write('{:0>2}_{} - {} ({})\n'.format(numer,metka[j],stime[j],speed))
                if star==1 and dol==1:
                    print('Ошибка в указаниии метки {}. Одновременно есть * и $'.format(metka[j]))
                    file_out.write('{:0>2}_{} - {} \n'.format(numer,metka[j],stime[j]))
                pictures.append(metka[j])
            else:
                for p in range(len(pictures)):
                    if metka[j]==pictures[p]:
                        break
                #file_out.write('{:0>2}_{} - {}\n'.format(p+1,metka[j],stime[j]))
                if star==0 and dol==0:
                    file_out.write('{:0>2}_{} - {}\n'.format(p+1,metka[j],stime[j]))
                if star==1 and dol==0:
                    speed=round((intervals[j]/SYSTEM_SIZE*1000),1)
                    if speed > 9.9:
                        print('\nРазмер картинки {} слишком мал. Ее длительность {} c Время {}'.format(metka[j],intervals[j],stime[j]))
                        print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                        speed=9.9
                        file_out.write('{:0>2}_{} - {} ({})\n'.format(p+1,metka[j],stime[j],speed))
                    if speed < 0.1:
                        print('\nРазмер картинки {} слишком велик. Ее длительность {} c. Время {}'.format(metka[j],intervals[j],stime[j]))
                        print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                        speed=0.1
                        file_out.write('{:0>2}_{} - {} ({})\n'.format(p+1,metka[j],stime[j],speed))
                    if speed >= 0.1 and speed <= 9.9:
                        file_out.write('{:0>2}_{} - {} ({})\n'.format(p+1,metka[j],stime[j],speed))
                if star==0 and dol==1:
                    speed=9.9
                    file_out.write('{:0>2}_{} - {} ({})\n'.format(p+1,metka[j],stime[j],speed))
                if star==1 and dol==1:
                    print(' \nОШИБКА в указаниии метки {}. Одновременно есть * и $\n'.format(metka[j]))
                    file_out.write('{:0>2}_{} - {} \n'.format(p+1,metka[j],stime[j]))
    
    #file_out.write(dop)
    file_out.close()
    print('Записано в '+ name_file_out)

if people==0:
    person1=[]
    rekv3=[]
    for t in range(len(persons)):
        if not persons[t]=='all':
            person1.append(persons[t])
            rekv3.append(rekv2[t])
    persons=person1
    rekv=rekv3
    file_out.write('Ищи в других файлах с номерами')
    file_out.close()
    per_n=-1
    for per in persons:
        kombo_pictures=[]
        per_n+=1
        numer=0
        name_path=direct(name_file_out+'_'+per)
        file_out=open(name_path+'/program.txt','w')
        for j in range(len(time)):
            if metka[j]=='end':
                dop='Finish - {}\nRepeat after finish - no\nLock buttons - yes'.format(stime[j])
                file_out.write(dop)
            else:
                flag_m=0
                kombo_metka=metka[j].split(';')
                kombo_person=person[j].split(';')
                for m in range(len(kombo_metka)):
                    star=0
                    dol = 0
                    if '*' in kombo_metka[m]:
                        kombo_metka[m] = kombo_metka[m].replace('*','')
                        star=1
                    if '$' in kombo_metka[m]:
                        kombo_metka[m] = kombo_metka[m].replace('$','')
                        dol=1

                    if per==kombo_person[m] or person[j]=='all':
                        if not kombo_metka[m] in kombo_pictures:
                            numer+=1
                            if rekv[per_n]==32:
                                ren32(path_pic,name_path,numer,rems(kombo_metka[m]))
                            if rekv[per_n]==64:
                                ren64(path_pic,name_path,numer,rems(kombo_metka[m]))
                            if not(rekv[per_n]==32) and not(rekv[per_n]==64):
                                print('Описка в указании длины реквизита. Неверное число пикселей')
                                print('Допустимое число пикселей - 32 или 64')
                            
                            #file_out.write('{:0>2}_{} - {}\n'.format(numer,kombo_metka[m],stime[j]))
                            if star==0 and dol==0:
                                file_out.write('{:0>2}_{} - {}\n'.format(numer,rems(kombo_metka[m]),stime[j]))
                            if star==1 and dol==0:
                                speed=round((intervals[j]/SYSTEM_SIZE*1000),1)
                                if speed > 9.9:
                                    print('\nРазмер картинки {} слишком мал. Ее длительность {} c. Время {}'.format(kombo_metka[m],intervals[j],stime[j]))
                                    print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                                    speed=9.9
                                if speed < 0.1:
                                    print('\nРазмер картинки {} слишком велик. Ее длительность {} c. Время {}'.format(kombo_metka[m],intervals[j],stime[j]))
                                    print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                                    speed=0.1
                                file_out.write('{:0>2}_{} - {} ({})\n'.format(numer,rems(kombo_metka[m]),stime[j],speed))
                            if star==0 and dol==1:
                                speed=9.9
                                file_out.write('{:0>2}_{} - {} ({})\n'.format(numer,rems(kombo_metka[m]),stime[j],speed))
                            if star==1 and dol==1:
                                print('Ошибка в указаниии метки {}. Одновременно есть * и $'.format(metka[j]))
                                file_out.write('{:0>2}_{} - {} \n'.format(numer,rems(kombo_metka[m]),stime[j]))
                            flag_m=1
                            kombo_pictures.append(kombo_metka[m])
                        else:
                            for p in range(len(kombo_pictures)):
                                if kombo_metka[m]==kombo_pictures[p]:
                                    break
                            #file_out.write('{:0>2}_{} - {}\n'.format(p+1,kombo_metka[m],stime[j]))
                            if star==0 and dol==0:
                                file_out.write('{:0>2}_{} - {}\n'.format(p+1,rems(kombo_metka[m]),stime[j]))
                            if star==1 and dol==0:
                                speed=round((intervals[j]/SYSTEM_SIZE*1000),1)
                                if speed > 9.9:
                                    print('\nРазмер картинки {} слишком мал. Ее длительность {} c. Время {}'.format(kombo_metka[m],intervals[j],stime[j]))
                                    print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                                    speed=9.9
                                if speed < 0.1:
                                    print('\nРазмер картинки {} слишком велик. Ее длительность {} c. Время {}'.format(kombo_metka[m],intervals[j],stime[j]))
                                    print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                                    speed=0.1
                                file_out.write('{:0>2}_{} - {} ({})\n'.format(p+1,rems(kombo_metka[m]),stime[j],speed))
                            if star==0 and dol==1:
                                speed=9.9
                                file_out.write('{:0>2}_{} - {} ({})\n'.format(p+1,rems(kombo_metka[m]),stime[j],speed))
                            if star==1 and dol==1:
                                print('Ошибка в указаниии метки {}. Одновременно есть * и $'.format(metka[j]))
                                file_out.write('{:0>2}_{} - {} \n'.format(p+1,rems(kombo_metka[m]),stime[j]))
                            flag_m=1
                if flag_m==0:
                    if not 'blackout' in kombo_pictures:
                        numer+=1
                        if rekv[per_n]==32:
                            ren32(path_pic,name_path,numer,'blackout')
                        if rekv[per_n]==64:
                            ren64(path_pic,name_path,numer,'blackout')
                        if not(rekv[per_n]==32) and not(rekv[per_n]==64):
                            print('Описка в указании длины реквизита. Неверное число пикселей')
                            print('Допустимое число пикселей - 32 или 64')
                        file_out.write('{:0>2}_{} - {}\n'.format(numer,'blackout',stime[j]))
                        kombo_pictures.append('blackout')
                    else:
                        for p in range(len(kombo_pictures)):
                                if 'blackout'==kombo_pictures[p]:
                                    break
                        file_out.write('{:0>2}_{} - {}\n'.format(p+1,'blackout',stime[j]))
        file_intervals.close()                
        file_out.close()
        print('Записано в '+name_file_out+'_'+per)
        

Не забудь поменять папку с картинками 

ВВедите имя будущей папки для реквизита. Помните, что оно должно начинаться с  XX_prog_
где XX - порядковый  номер программы у пойки
11_prog_routable
Успешно создана дирректория /home/gatupov/python/ignis_projects/ignis/11_prog_routable
Введите имя дорожки отметок
routable_wave.txt

['blackout', '*wavebl', '*wavebl', '*wavebl', '*wavebl', '*wavebl', '*wavebl', '*wavebl', 'yellow1', 'blue1', 'green1', 'blue1;blue1', 'red1;red1', 'galka;galka', 'violet1', 'red1', 'purple1', 'end']
['3.991117', '6.482285', '6.930627', '7.401411', '8.131128', '8.615364', '9.092874', '9.762062', '10.230897', '13.980518', '17.731085', '19.625011', '21.423235', '29.072371', '34.615926', '36.481274', '43.989096', '51.478749']
['1d', '2d', '3d', '4d']
[32, 64, 32, 32]
['all', '1d', '2d', '3d', '4d']
['00:00:00', '00:01:49', '00:01:93', '00:02:41', '00:03:14', '00:03:62', '00:04:10', '00:04:77', '00:05:23', '00:08:98', '00:12:73', '00:14:63', '00:16:43', '00:24:08', '00:29

In [1]:
print(floor(5.3))

NameError: name 'floor' is not defined